In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.appName('String_dates').getOrCreate()

In [6]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [7]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [8]:
emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [11]:
# Case When
# select employee_id, name, age, salary, gender,
# case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end as new_gender, hire_date from emp


emp_gender_fixed = emp.withColumn('new_gender', when (col('gender')=='Male','M')
                                                .when (col('gender')=='Female','F')
                                                 .otherwise(None))

emp_gender_fixed_1 = emp.withColumn("new_gender",expr("case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end"))

In [12]:
emp_gender_fixed_1.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M|
+-----------+-------------+----------+---+------+------+----------+----------+
only showing top 5 rows



In [14]:
# Replace in Strings
# select employee_id, name, replace(name, 'J', 'Z') as new_name, age, salary, gender, new_gender, hire_date from emp_gender_fixed

emp_name_fixed = emp_gender_fixed.withColumn('new_name',regexp_replace('name','J','Z'))

In [16]:
emp_name_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim

In [18]:
# Convert Date
# select *,  to_date(hire_date, 'YYYY-MM-DD') as hire_date from emp_name_fixed

emp_name_fixed.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [19]:
emp_date_fix = emp_name_fixed.withColumn('hire_date',to_date('hire_date','yyyy-MM-dd'))

In [21]:
emp_date_fix.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [24]:
new_columns ={
'date_now': current_date(),
'timestamp_now':current_timestamp()
}

In [25]:
emp_dated = emp_date_fix.withColumns(new_columns)

In [28]:
emp_dated.show(truncate=False)

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |new_gender|new_name     |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|M         |Zohn Doe     |2024-09-19|2024-09-19 15:18:23.440546|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|F         |Zane Smith   |2024-09-19|2024-09-19 15:18:23.440546|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|M         |Bob Brown    |2024-09-19|2024-09-19 15:18:23.440546|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|F         |Alice Lee    |2024-09-19|2024-09-19 15:18:23.440546|
|005        |103          |Jack Chan    |40 |Male  |60000 |201

In [29]:
# Drop Null gender records
emp_1 = emp_dated.dropna()


In [30]:
# 2. Drop rows where specific columns contain null:
# emp_dated = emp_dated.dropna(subset=["column1", "column2"])


# 3. Drop rows where all columns are null:
# emp_dated = emp_dated.dropna(how="all")


# 4. Drop rows where threshold number of non-null values are not met:
# emp_dated = emp_dated.dropna(thresh=2)  # Keep rows with at least 2 non-null values


In [32]:
# Fix Null values
# select *, nvl('new_gender', 'O') as new_gender from emp_dated

emp_null_df= emp_dated.withColumn('new_gender',coalesce(col('new_gender'),lit('o')))

In [33]:
emp_null_df.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2024-09-19|2024-09-19 15:30:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2024-09-19|2024-09-19 15:30:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2024-09-19|2024-09-19 15:30:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2024-09-19|2024-09-19 15:30:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2024-09-1

In [34]:
emp_final = emp_null_df.drop("name", "gender").withColumnRenamed()

In [35]:
column_renames = {"name": "new_name", "gender": "new_gender"}

In [36]:
for old_name,new_name in column_renames.items():
    emp_final = emp_final.withColumnRenamed(old_name,new_name)

In [38]:
emp_final.show()

+-----------+-------------+---+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|age|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+---+------+----------+----------+-------------+----------+--------------------+
|        001|          101| 30| 50000|2015-01-01|         M|     Zohn Doe|2024-09-19|2024-09-19 15:36:...|
|        002|          101| 25| 45000|2016-02-15|         F|   Zane Smith|2024-09-19|2024-09-19 15:36:...|
|        003|          102| 35| 55000|2014-05-01|         M|    Bob Brown|2024-09-19|2024-09-19 15:36:...|
|        004|          102| 28| 48000|2017-09-30|         F|    Alice Lee|2024-09-19|2024-09-19 15:36:...|
|        005|          103| 40| 60000|2013-04-01|         M|    Zack Chan|2024-09-19|2024-09-19 15:36:...|
|        006|          103| 32| 52000|2018-07-01|         F|    Zill Wong|2024-09-19|2024-09-19 15:36:...|
|        007|          101| 42| 70000

In [40]:
emp_final.repartition(1).write.mode('overwrite').option('header',True).format("csv").save("san")